In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 49.0 MB/s eta 0:00:00


In [9]:
import torch
import torch.onnx
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import os

# 1. 모델과 토크나이저 로드 (kobert-base)
model_name = "klue/bert-base"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

model.eval()

# 2. ONNX 파일 및 토크나이저 파일 저장 경로 설정 (Google Drive)
output_dir = "/content/drive/MyDrive/onnx_model_for_android"
os.makedirs(output_dir, exist_ok=True)

onnx_model_path = os.path.join(output_dir, "bert.onnx")
vocab_path = os.path.join(output_dir, "vocab.txt")

# 3. 더미 입력 생성
dummy_text = "안녕하세요."
dummy_input_dict = tokenizer(
    dummy_text,
    return_tensors="pt",
    padding='max_length',
    max_length=128, # 안드로이드에서 사용할 최대 시퀀스 길이와 맞추는 것이 좋음
    truncation=True
)

# 4. ONNX로 변환
# 입력 텐서의 이름과 출력 텐서의 이름
input_names = ['input_ids', 'attention_mask']
output_names = ['logits']

# 딕셔너리 형태의 더미 입력을 ONNX 변환을 위한 '튜플'로 변환
args_tuple = (
    dummy_input_dict['input_ids'],
    dummy_input_dict['attention_mask']
)

print(f"ONNX 모델을 {onnx_model_path} 경로로 내보냅니다.")
torch.onnx.export(
    model,
    args_tuple,
    onnx_model_path,
    verbose=False,
    input_names=input_names,
    output_names=output_names,
    opset_version=14,
    dynamic_axes={ # 배치 크기와 시퀀스 길이를 동적으로 처리
        'input_ids': {0: 'batch_size', 1: 'sequence_length'},
        'attention_mask': {0: 'batch_size', 1: 'sequence_length'},
        'logits': {0: 'batch_size'}
    }
)

# 5. 토크나이저의 vocab.txt 저장 (안드로이드에서 사용)
tokenizer.save_vocabulary(output_dir)

print(f"✅ KoBERT ONNX 모델 '{os.path.basename(onnx_model_path)}'와 'vocab.txt'가 '{output_dir}'에 성공적으로 생성되었습니다!")
print(f"생성된 파일: {os.path.abspath(output_dir)}")

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

ONNX 모델을 /content/drive/MyDrive/onnx_model_for_android/kobert_sequence_classification.onnx 경로로 내보냅니다.


/tmp/ipython-input-2872421905.py:47: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(


✅ KoBERT ONNX 모델 'kobert_sequence_classification.onnx'와 'vocab.txt'가 '/content/drive/MyDrive/onnx_model_for_android'에 성공적으로 생성되었습니다!
생성된 파일: /content/drive/MyDrive/onnx_model_for_android
